## 1. 生成訓練資料集與測試資料集

### Training data

#### 原先訓練集資料只有 question 和 answer ，因此我下載含有 triple 的訓練資料集。
#### 來源 : https://github.com/jkszw2014/bert-kbqa-NLPCC2017/blob/master/NER_BERT-BiLSTM-CRF/NLPCC2016KBQA/nlpcc-iccpol-2016.kbqa.training-data

In [1]:
# coding:utf-8
import sys
import os
import pandas as pd
import numpy as np

data_type = "training"
file = "nlpcc-iccpol-2016.kbqa."+data_type+"-data"
question_str = "<question"
triple_str = "<triple"
answer_str = "<answer"
start_str = "============="


q_t_a_list = []
relation_list = []
seq_q_list = []
seq_tag_list = []

with open(file,'r', encoding="utf-8") as f:
    q_str = ""
    t_str = ""
    a_str = ""

    for line in f:
        if question_str in line:
            q_str = line.strip()
        if triple_str in line:
            t_str = line.strip()
        if answer_str in line:
            a_str = line.strip()

        if start_str in line: #new question answer triple
            entities = t_str.split("|||")[0].split(">")[1].strip()
            relation = t_str.split("||| ")[1].strip()
            q_str = q_str.split(">")[1].replace(" ","").strip()
            if entities in q_str:
                q_list = list(q_str)
                seq_q_list.extend(q_list)
                seq_q_list.extend([" "])
                
                tag_list = ['O' for i in range(len(q_list))]
                tag_start_index = q_str.find(entities)
                for i in range(tag_start_index, tag_start_index+len(entities)):
                    if tag_start_index == i:
                        tag_list[i] = 'B-LOC'
                    else:
                        tag_list[i] = 'I-LOC'
                seq_tag_list.extend(tag_list)
                seq_tag_list.extend([" "])
                
            else:
                pass
            q_t_a_list.append([q_str.encode("utf-8"), t_str, a_str])
            relation_list.append(relation)

In [2]:
NER_training_data = pd.DataFrame(columns = ['question', 'tag' , 'entity', 'purpose', 'relation'])

k = 0

Q, T = [], []

j = 0

for i in range(len(seq_q_list)) :
     
    if seq_q_list[i] == ' ' :
        
        P = np.array(Q)[np.array(T) == 'O'].tolist()
        E = np.array(Q)[np.array(T) != 'O'].tolist()
        E = ''.join(E)
        
        Q = [Q]
        Q.append(T)
        Q.append(E)
        Q.append(P)
        Q.append(relation_list[j])
        
        j += 1
        
        NER_training_data.loc[k] = Q
        
        Q, T = [], []
        
        k += 1
        
    else :
        
        Q.append(seq_q_list[i])
        
        T.append(seq_tag_list[i])

In [3]:
NER_training_data.head()

,question,tag,entity,purpose,relation
0,"[《, 机, 械, 设, 计, 基, 础, 》, 这, 本, 书, 的, 作, 者, 是, ...","[O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, ...",机械设计基础,"[《, 》, 这, 本, 书, 的, 作, 者, 是, 谁, ？]",作者
1,"[《, 高, 等, 数, 学, 》, 是, 哪, 个, 出, 版, 社, 出, 版, 的, ？]","[O, B-LOC, I-LOC, I-LOC, I-LOC, O, O, O, O, O,...",高等数学,"[《, 》, 是, 哪, 个, 出, 版, 社, 出, 版, 的, ？]",出版社
2,"[《, 线, 性, 代, 数, 》, 这, 本, 书, 的, 出, 版, 时, 间, 是, ...","[O, B-LOC, I-LOC, I-LOC, I-LOC, O, O, O, O, O,...",线性代数,"[《, 》, 这, 本, 书, 的, 出, 版, 时, 间, 是, 什, 么, ？]",出版时间
3,"[安, 德, 烈, 是, 哪, 个, 国, 家, 的, 人, 呢, ？]","[B-LOC, I-LOC, I-LOC, O, O, O, O, O, O, O, O, O]",安德烈,"[是, 哪, 个, 国, 家, 的, 人, 呢, ？]",国籍
4,"[《, 线, 性, 代, 数, 》, 的, i, s, b, n, 码, 是, 什, 么, ？]","[O, B-LOC, I-LOC, I-LOC, I-LOC, O, O, O, O, O,...",线性代数,"[《, 》, 的, i, s, b, n, 码, 是, 什, 么, ？]",isbn


In [4]:
NER_training_data.to_csv('04_NER_training_data.csv', index = False, encoding='utf_8_sig')

### Testing data

In [5]:

file = '04_Kbqa.testing-data'
question_str = "<question"
triple_str = "<triple"
answer_str = "<answer"
start_str = "============="

q_t_a_list = []
seq_q_list = []
seq_tag_list = []

with open(file,'r', encoding="utf-8") as f:
    q_str = ""
    t_str = ""
    a_str = ""

    for line in f:
        if question_str in line:
            q_str = line.strip()
        if triple_str in line:
            t_str = line.strip()
        if answer_str in line:
            a_str = line.strip()

        if start_str in line: #new question answer triple
            q_str = q_str.split(">")[1].replace(" ","").strip()
            seq_q_list.append(list(q_str))

In [6]:
NER_testing_data = pd.DataFrame({'question':seq_q_list})

In [7]:
NER_testing_data.head()

,question
0,"[你, 知, 道, 计, 算, 机, 应, 用, 基, 础, 这, 本, 书, 的, 作, ..."
1,"[计, 算, 机, 应, 用, 基, 础, 这, 本, 书, 的, 出, 版, 社, 是, ..."
2,"[告, 诉, 我, 高, 等, 数, 学, 的, 出, 版, 时, 间, 是, 什, 么, ..."
3,"[我, 想, 知, 道, 戴, 维, 斯, 是, 什, 么, 国, 家, 的, 人, ？]"
4,"[你, 知, 道, 高, 等, 数, 学, 的, i, s, b, n, 吗, ？]"


In [8]:
NER_testing_data.to_csv('04_NER_testing_data.csv', index = False, encoding='utf_8_sig')